#Advent of Code 2017

My Solutions to 2017 Advent of Code (http://adventofcode.com) in Python.

They may not be all perfect but they work!

##Setup

###Load Needed Libraries

In [1]:
import math
import numpy as np
import operator
from collections import Counter

####Function for Loading Puzzle Inputs

In [39]:
def load_data(day):
    filename = "input/"+ day + "-input.txt"

    try:
        return open(filename)
    except FileNotFoundError:
        print("Failed to load file")

##Day 1: Inverse Captcha

####Part1:

To solve I create a rotated version of the input string.  We can than iterate and add any numbers which are in the same position in the original and rotated list


In [40]:
def captcha(text,rotate):
    text_r = text[rotate:] + text[:rotate]
    total = 0
    
    for i in range(0,len(text)):
        if text_r[i] == text[i]:
            total += int(text[i])
            
    return total

#test
assert captcha("1122",1) == 3
assert captcha("1111",1) == 4
assert captcha("1234",1) == 0
assert captcha("91212129",1) ==9
captcha(load_data("day1").read(),1)

1089

###Part 2

Instead of rotating by one, rotate by half the length of the input.

In [41]:
#test
assert captcha("1212",2) == 6
assert captcha("1221",2) == 0
assert captcha("123425",3) == 4
assert captcha("123123",3) ==12
assert captcha("12131415",4) ==4

r = int(len(load_data("day1").read())/2)
captcha(load_data("day1").read(),r)

1156

##Day 2: Corruption Checksum

###Part 1:

The solution for this is mostly about correctly formatting the data. I split the input into a list of lists.  The lower level lists are each row and the higher level list is the entire spreadsheet.  We than count the difference of each rows max and min.

In [42]:
def checksum(spreadsheet):
    total = 0
    
    for row in spreadsheet:
        total += (max(row) - min(row))
        
    return total

def load_ss():
    input = []
    for line in load_data("day2"):
        t = line.rstrip().split("\t")
        t = list(map(int, t))
        input.append(t)
    return input

assert checksum([[5,1,9,5],[7,5,3],[2,4,6,8]]) == 18

checksum(load_ss())

45972

###Part 2:

Use the same data structure as for part one.  However this time we need to iterate through each row to find which two elements in each row are divisible.


In [43]:
def get_divisor(row):
    
    for i in range(0,len(row)-1):
        for j in range(i+1,len(row)):
            if row[i]%row[j] == 0.0:
                return int(row[i]/row[j])
            elif row[j]%row[i] == 0.0:
                return int(row[j]/row[i])
    return 0
    

def checksum_2(spreadsheet):
    total = 0
    
    for row in spreadsheet:
        total += get_divisor(row)
        
    return total
    
assert checksum_2([[5,9,2,8],[9,4,7,3],[3,8,6,5]]) == 9

checksum_2(load_ss())

326

##Day 3: Spiral Memory

###Part 1:

The bottom right hand corner of each square is the square of odd numbers (ie 9, 25, 49, 81) we can use this to determine which square the specified number is located in.


In [44]:
puzzle_input_day2 = 361527

def spiral_dist(num):
    square = 1
    x = 0
    y = 0
    
    #if its already the centre
    if num == 1:
        return 0
    
    #determine which square the number is in
    #ie square is the size of square sides that the square containing the number will have
    while True:
        square += 2
        if num <= square ** 2:
            break
            
    #this is teh value in bottom right corner of the square
    #it is the biggest number in the square containing the number of interest
    corner = square ** 2
    
    #calculate number of squares or layers before current square
    #no matter what we must travel this distance
    dist = math.floor(square/2)
    
    #calculate distance from nearest mid point in square side
    #we will need to travel this distance in addtion to the number of squares before current square
    dist += min([abs(num - (corner - dist)), abs(num - (corner - 3 * dist)),
                 abs(num - (corner - 5 * dist)),abs(num - (corner - 7 * dist))])
    
    return dist


assert spiral_dist(1) == 0
assert spiral_dist(12) == 3
assert spiral_dist(23) == 2
assert spiral_dist(1024) == 31

spiral_dist(puzzle_input_day2)

326

###Part 2:


Try brute forcing using numpy.  Create the spiral in a numpy array and not when we encounter a first number above the puzzle input.

In [45]:
#generate square
size = 32
global day2_match
day2_match = False

def sum_neighbours(x,y):
    global day2_match
    square[x,y] = square[x-1,y] + square[x+1,y] + square[x,y-1] + square[x,y + 1] + square[x-1,y - 1] + square[x-1,y +1] + square[x + 1,y-1] + square[x + 1,y + 1]
    
    if (square[x,y] > puzzle_input_day2) and day2_match == False:
        print("Answer is: " + str(int(square[x,y])))
        day2_match = True
    return

def add_square(num):
    x =  size//2 + num
    y =  size//2 + num
    
    #right side
    for i in range(0,(num*2)):
        x -= 1
        sum_neighbours(x,y)
        
    #top
    for j in range(0,(num*2)):
        y -= 1
        sum_neighbours(x,y)
    
    #left
    for k in range(0,(num*2)):
        x += 1
        sum_neighbours(x,y)

    #bottom
    for m in range(0,(num*2)):
        y += 1
        sum_neighbours(x,y)
    
    return
    
   
#initalize array and set center to 1
square = np.zeros((size,size))
square[size//2,size//2] = 1

#add squares
for i in range (1,5):
    add_square(i)

Answer is: 363010


##Day 4: High-Entropy Passphrases

###Part 1:

Approach is to create a set of each line and add each word one by one.  If word already in set break and don't count that passcode.

In [46]:
def valid_passcode(passcode):
    pc_set = set()
    for word in passcode:
        if word in pc_set:
            return 0
        else:
            pc_set.add(word)
    return 1

pc_count = 0

for line in load_data("day4"):
    pc = line.rstrip().split(" ")
    pc_count += valid_passcode(pc)

assert valid_passcode("aa bb cc dd ee".split(" ")) == 1
assert valid_passcode("aa bb cc dd aa".split(" ")) == 0
assert valid_passcode("aa bb cc dd aaa".split(" ")) == 1

pc_count

383

###Part 2:

Solve using a dict to compare pairs of strings and see if they are anagrams.  Repeat for every possible pairing of words in each passcode.

In [47]:
def is_anagram(word1,word2):
    #add each letter in word 1 to dict
    #subtract each letter in word 2
    #if the dict values sum to 0 it must be an anagram
    letter_dict1 = {}
    letter_dict2 = {}
    
    for letter in word1:
        letter_dict1[letter] = letter_dict1.get(letter, 0) + 1
        
    for letter in word2:
        letter_dict2[letter] = letter_dict2.get(letter, 0) + 1
        
    if letter_dict1 == letter_dict2:
        return True

    return False

#use is_anagram to compare all pairs in string for anagrams
def valid_passcode_anagram(passcode):
    for i in range(0,len(passcode)-1):
        for j in range(i+1,len(passcode)):
            if is_anagram(passcode[i],passcode[j]):
                return 0
    return 1

#valid_passcode_anagram("abcde xyz ecdab".split(" "))

assert valid_passcode_anagram("abcde fghij".split(" ")) == 1
assert valid_passcode_anagram("abcde xyz ecdab".split(" ")) == 0
assert valid_passcode_anagram("a ab abc abd abf abj".split(" ")) == 1
assert valid_passcode_anagram("iiii oiii ooii oooi oooo".split(" ")) == 1
assert valid_passcode_anagram("oiii ioii iioi iiio".split(" ")) == 0

pc_count_anagram = 0
for line in load_data("day4"):
    pc = line.rstrip().split(" ")
    pc_count_anagram += valid_passcode_anagram(pc)

pc_count_anagram

265

##Day 5: A Maze of Twisty Trampolines, All Alike

###Part 1:

Create array with instructions and update as required until we jump "outside" of the array.

In [48]:
#load instructions into a list of integers
instructions = load_data("day5").read()
instructions = instructions.rstrip().split("\n")
instructions = list(map(int,instructions))

#count jumps
def count_jumps(instructions):
    ptr = 0
    counter = 0
    length = len(instructions)
    while True:
        instruct = instructions[ptr]
        instructions[ptr] += 1
        ptr += instruct
        counter += 1
        if ptr > length - 1:
            return counter
        
assert count_jumps([0,3,0,1,-3]) == 5

count_jumps(instructions)

356945

###Part 2:

Same as previous solution but update the offsets.  It runs a little slower!

In [49]:
#reload instructions into a list of integers
instructions = load_data("day5").read()
instructions = instructions.rstrip().split("\n")
instructions = list(map(int,instructions))


#count jumps
def count_jumps_p2(instructions):
    ptr = 0
    counter = 0
    length = len(instructions)
    while True:
        instruct = instructions[ptr]
        
        if instructions[ptr] >= 3:
            instructions[ptr] -= 1
        else:
            instructions[ptr] += 1
            
        ptr += instruct
        counter += 1
        if ptr > length - 1:
            return counter
        
assert count_jumps_p2([0,3,0,1,-3]) == 10

count_jumps_p2(instructions)

28372145

##Day 6: Memory Reallocation

###Part 1:

Loop through the array incrementing the registers as required.  Use a set to track the previous sets we have encountered.  Return the answer once a match to the set is found.


In [50]:
#load inital registers
reg = load_data("day6").read()
reg = reg.rstrip().split("\t")
reg = list(map(int,reg))

def mem_reallocate(reg):

    #set of registers we have seen
    past_reg = set()
    past_reg.add("-".join(str(x) for x in reg))
    
    #step count 
    counter = 0

    while True:
        max_i = reg.index(max(reg))
        max_v = reg[max_i]
        reg[max_i] = 0
        
        for i in range(1, max_v+1):
            reg[(max_i+i)%16] += 1
            
        counter += 1
        
        new_reg_str = "-".join(str(x) for x in reg)
        
        if new_reg_str in past_reg:
            return counter, reg
        
        past_reg.add(new_reg_str)
    
ans, new_reg = mem_reallocate(reg)

print("The answer is: ",str(ans))

The answer is:  12841


###Part 2:
Use the answer from part 1.  Run until we get a match to this register.

In [51]:

def mem_reallocate_p2(reg):

    #set of registers we have seen
    past_reg = "-".join(str(x) for x in reg)
    
    #step count 
    counter = 0

    while True:
        max_i = reg.index(max(reg))
        max_v = reg[max_i]
        reg[max_i] = 0
        
        for i in range(1, max_v+1):
            reg[(max_i+i)%16] += 1
            
        counter += 1
        
        new_reg_str = "-".join(str(x) for x in reg)
        
        if new_reg_str == past_reg:
            return counter

mem_reallocate_p2(new_reg)

8038

##Day 7: Recursive Circus

###Part 1:

We need to load data into two dicts one with the node weights and one with children of the nodes.  We can then use recurision to determine the depth of tree below each node.  The node with the greatest depth below it will be the root.

In [52]:
#load data
weights = {}
children = {}

#load data into two dicts, one for weights and 1 for children
for line in load_data("day7"):
    
        # add weight to weights dict
        node = line.rstrip().split(" ")[0]
        weight = line.rstrip().split(" ")[1]
        weight = weight[weight.find("(")+1:weight.find(")")]
        weights[node] = weight
        
        #add children to children dict
        if line.count(" ") > 1:
            children_raw = line.rstrip().split("-> ")[1]
            children_raw = children_raw.strip().split(",")
            children[node] = [child.strip() for child in children_raw]

def depth(node,level):
    if node not in children.keys():
        return level + 1
    for child in children[node]:
        return depth(child,level+1)

max_depth = 0
root = ""

#find node with greatest depth below it
for node in weights.keys():
    node_d = depth(node,0)
    if node_d > max_depth:
        max_depth = node_d
        root = node
        
print("Max depth: " + str(max_depth))
print("Root: " + root)


Max depth: 6
Root: veboyvy


###Part 2:
Calculate the subtree weights of all children of the root node.  One node will be unbalanced.  Than recursively call again on the unbalanced node.  Repeat until nodes are balanced.  This node that has all balanced children but was in a unbalanced branch must be imbalanced one.  We can than subtract the needed correction from it's known weight.

In [53]:
def subtree_weight(node):
    if node not in children.keys():
        return int(weights[node])
    return int(weights[node]) + sum([subtree_weight(child) for child in children[node]])

def find_unbalanced_node(node):
    weights = []
    
    for child in children[node]:
        weights.append(subtree_weight(child))
    

    if len(set(weights)) == 1:
        return node
    
    else:
        index_offbalance = weights.index([x for x in weights if weights.count(x)==1][0])
        return find_unbalanced_node(children[node][index_offbalance])

    
unbalanced = find_unbalanced_node('veboyvy')
correct_weight = int(weights[unbalanced]) - 7
print("Unbalanced node: " + str(unbalanced))
print("New weight is: " + str(correct_weight))

Unbalanced node: obxrn
New weight is: 749


##Day 8: I Heard You Like Registers

###Part 1/Part 2:

Create a queue of commands and pop them off one by one.  Use a dict to store the values of the registers.  Can modify registers in place using process_commands function.

Modified process_commands to track the largest register ever encountered for part 2 of the question.

In [54]:
registers = {}
commands = []
max_register_ever = -9999

commands_test = ["b inc 5 if a > 1",
"a inc 1 if b < 5",
"c dec -10 if a >= 1",
"c inc -20 if c == 10"]

#load data
for line in load_data("day8"):
    commands.append(line.rstrip())

#prepare register    
def init_register(register):
        if register not in registers.keys():
            registers[register] = 0

#check if command operation is true            
def condition_is_true(register,operator,value):
    if operator == "!=":
        return registers[register] != int(value)
    
    elif operator == "==":
        return registers[register] == int(value)
    
    elif operator == "<=":
        return registers[register] <= int(value)
    
    elif operator == ">=":
        return registers[register] >= int(value)
    
    elif operator == "<":
        return registers[register] < int(value)
    
    elif operator == ">":
        return registers[register] > int(value)
 
#perform the command operation on register   
def perform_operation(operand_register,operation,increment):
    if operation =="inc":
        registers[operand_register] += int(increment)
        
    elif operation =="dec":
        registers[operand_register] -= int(increment)

#run through queue of commands and perform them
#return largest ever register for part 2
def process_commands(commands):
    max_register_ever = -9999
    
    while commands:
        operand_register,operation,increment,condition,cond_register,cond_opt,cond_amount = commands.pop(0).rstrip().split(" ")
        
        init_register(cond_register)
        init_register(operand_register)
        
        if condition_is_true(cond_register,cond_opt,cond_amount):
            perform_operation(operand_register,operation,increment)
        
        if max(registers.values()) > max_register_ever:
            max_register_ever = max(registers.values())
            
    return max_register_ever
            
max_reg = process_commands(commands)
print("Largest final register: " + str(max(registers.values())))
print("Largest ever register: " + str(max_reg))

Largest final register: 7787
Largest ever register: 8997


##Day 9: Stream Processing

###Part 1/Part 2:

Build a stack and add to it everytime we encounter a "{" and pop from the stack everytime we encounter "}".  If we see a "<" go into garbage mode and ignore brackets.  If we see a "!" ignore the next letter by removing it from the garbage stream.

In [55]:
def process_garbage(stream):
    stack = []
    count = 0
    garbage = False
    str = list(stream)
    garbage_count = 0
    while str:
        c = str.pop(0)
        
        if c == "!":
            str.pop(0)
            
        elif c != "!" and c != ">" and garbage:
            garbage_count += 1
            
        if not garbage:
            if c == "{":
                stack.append("}")
            elif len(stack) and c == stack[-1]:
                stack.pop()
                count += len(stack) + 1
            elif c == "<":
                garbage = True
                       
        elif c == ">":
            garbage = False
            
    return [count,garbage_count]

assert process_garbage("{}")[0] == 1
assert process_garbage("{{{}}}")[0] == 6
assert process_garbage("{{},{}}")[0] == 5
assert process_garbage("{{{},{},{{}}}}")[0] == 16
assert process_garbage("{<a>,<a>,<a>,<a>}")[0] == 1
assert process_garbage("{{<ab>},{<ab>},{<ab>},{<ab>}}")[0] == 9
assert process_garbage("{{<!!>},{<!!>},{<!!>},{<!!>}}")[0] == 9
assert process_garbage("{{<a!>},{<a!>},{<a!>},{<ab>}}")[0] == 3

#load data
stream = load_data("day9").read().rstrip()
process_garbage(stream)

[9251, 4322]

##Day 10: Knot Hash

###Part 1:

Use a list and modify the number ordering per the instructions.

In [56]:
def knot_hash(elements,lengths):
    #intialize
    l = [x for x in range(0,elements)]
    skip_size = 0
    ptr = 0
    
    for length in lengths:
        
        rev_l = []
        #get elements
        for i in range(0,length):
            rev_l.append(l[(ptr+i)%elements])
            
        #reverse
        rev_l = rev_l[::-1]
        
        #replace in list
        for i in range(0,length):
            l[(ptr+i)%elements] =rev_l[i]
        
        ptr += length + skip_size
        skip_size += 1

    return l[0] * l[1]
        
    
    
assert knot_hash(5,[3,4,1,5]) == 12

lengths = load_data('day10').read().rstrip().split(",")
lengths = list(map(int,lengths))

knot_hash(256,lengths)

23715

###Part 2:

Gets a little trickier here.  Modify the function from part 1 to generate the sparse hash of the input. Dense hash than converts the sparse hash into a fixed length hexidecimal dense hash.

In [57]:
#convert input to ascii codes
lengths = load_data('day10').read().rstrip()

def sparse_hash(elements,lengths):
    #intialize
    lengths = [ord(c) for c in lengths]
    lengths = lengths + [17, 31, 73, 47, 23]

    l = [x for x in range(0,elements)]
    skip_size = 0
    ptr = 0
    
    for j in range(0,64):
        for length in lengths:
            
            rev_l = []
            #get elements
            for i in range(0,length):
                rev_l.append(l[(ptr+i)%elements])
                
            #reverse
            rev_l = rev_l[::-1]
            
            #replace in list
            for i in range(0,length):
                l[(ptr+i)%elements] =rev_l[i]
            
            ptr += length + skip_size
            skip_size += 1

    return l

def dense_hash(sparse_hash):
    dense=[]
    hex_string = []
    for i in range(0,16):
        t = sparse_hash[(16*i):(16*i + 16)]
        dt = t[0]
        for x in t[1:16]:
            dt = dt ^ x
        dense.append(dt)
        
    #convert to hex
    for c in dense:
        hex_c = '{:x}'.format(c)
        #correct due to problem with hex not putting 0 in front of single digit nums
        if len(hex_c) == 1:
            hex_c = "0" + hex_c
        hex_string.append(hex_c)
        
    return "".join(hex_string)


assert dense_hash(sparse_hash(256,""))  == "a2582a3a0e66e6e86e3812dcb672a272"
assert dense_hash(sparse_hash(256,"AoC 2017"))  == "33efeb34ea91902bb2f59c9920caa6cd"
assert dense_hash(sparse_hash(256,"1,2,3")) == "3efbe78a8d82f29979031a4aa0b16a9d" 
assert dense_hash(sparse_hash(256,"1,2,4"))  == "63960835bcdc130f0b66d7ff4f6a5a8e" 

dense_hash(sparse_hash(256,lengths))

'541dc3180fd4b72881e39cf925a50253'

##Day 11: Hex Ed

###Part 1:

Solved using cube dimensions for hexes described here: https://www.redblobgames.com/grids/hexagons/

In [59]:
def fewest_steps(steps):
    x = 0
    y = 0
    z = 0
    for step in steps:
        
        if step =="nw":
            y += 1
            x += -1
            
        elif step == "n":
            y += 1
            z += -1
            
        elif step == "ne":
            x += 1
            z += -1
        
        elif step == "se":
            x += 1
            y += -1
        
        elif step == "s":
            y -= 1
            z += 1
            
        elif step == "sw":
            x += -1
            z += 1
            
    return max(abs(x),abs(y),abs(z))


assert fewest_steps(['ne','ne','ne'])  == 3
assert fewest_steps(['ne','ne','sw','sw'])  == 0
assert fewest_steps(['ne','ne','s','s'])  == 2
assert fewest_steps(['se','sw','se','sw','sw'])  == 3

fewest_steps(load_data('day11').read().rstrip().split(","))

810

###Part 2:
Using the function from part 1, we just iterate through all the steps one by one and record the longest distance ever from the origin.

In [60]:
steps = load_data('day11').read().rstrip().split(",")
max_dist = 0

for i in range(0,len(steps)):
    max_dist = max(max_dist,fewest_steps(steps[0:i]))
    
print(max_dist)

1567


##Day 12: Digital Plumber

###Part 1:
Build up the input graph in a dict.  Than use DFS to traverse starting at program/node 0 and count how many different nodes we visit.

In [61]:
#load data and create graph
rg = load_data('day12').read().rstrip().split("\n")
g = {}

for path in rg:
    node = int(path.split(" <-> ")[0])
    connections = path.split(" <-> ")[1].split(",")
    connections = [int(x.strip(' ')) for x in connections]
    
    g[node] = connections
    
#dfs the graph to find everything connected to program "0"
def find_group(g,start_program):
    visited = set()
    stack = [start_program]
    while stack:
        program = stack.pop()
        if program not in visited:
            visited.add(program)
            for connection in g[program]:
                stack.append(connection)
                
    return visited

len(find_group(g,0))

141

###Part 2:
Iteratively call find_group on the list of the nodes.  After each run remove nodes that were visited and then re-run on one of the remaining nodes.  Count the number of time this is repeated until no nodes remain to find the groupings.

In [62]:
nodes = list(g.keys())
groups = 0

while nodes:
    visited = list(find_group(g,nodes.pop(0)))
    groups += 1
    nodes = [x for x in nodes if x not in visited]
    
print(groups)

171


##Day 13: Packet Scanners

###Part 1:
Create a dict to store firewall and its layers using load_fw().  Than cycle through the firewall for each second.  If scanner is at 0 for that scanner increase severity.

In [63]:
#load and prep firewall
def load_fw(fw_input):
    fw_input = fw_input.rstrip().split("\n")
    fw = {}
    
    #fw structure [depth,scanner_pos, update direction
    for step in fw_input:
        fw[int(step.split(": ")[0])] = int(step.split(": ")[1])
    
    for i in range(0,max(fw.keys())):
        if i not in fw.keys():
            fw[i] = 0
            
    return fw

def calc_severity(firewall):
    severity = 0
    
    for ps in range(0,max(firewall.keys())+1):
        layer_range = firewall[ps]
        #if no scanner
        if layer_range == 0:
            continue
        #determine if the scanner is position zero when the scanner
        #cross through this layer
        if (ps % ((layer_range-1)*2)) == 0:
            severity += ps * layer_range
            
    return severity
        
        
    
assert calc_severity(load_fw("0: 3\n1: 2\n4: 4\n6: 4")) == 24

calc_severity(load_fw(load_data('day13').read()))

1704

###Part 2:

Modify the severity calculator from part 1.  Keepy increasing delay by one picosecond until the pointer is not caught.  Still a more or less brute force solution.

In [64]:
def calc_delay(firewall):

    #iterate through different delay amounts
    for delay in range(0,4000000):
        caught = False
        
        #cycle through with the given delay to see if pointer is caught.
        for ps in range(0,max(firewall.keys())+1):
            layer_range = firewall[ps]
            
            #if no scanner
            if layer_range == 0:
                continue
            
            #determine if the scanner is position zero when the scanner
            #cross through this layer, added in the delay components   
            if ((ps+delay) % ((layer_range-1)*2)) == 0:
                caught = True
                
        if caught == False:
            return delay        
    return 0

assert calc_delay(load_fw("0: 3\n1: 2\n4: 4\n6: 4")) == 10

calc_delay(load_fw(load_data('day13').read()))

3970918

##Day 14: Disk Defragmentation

###Part 1:

We can use the hashing functions from day 10 to generate the hash.  We can than iterate through the hash to generate the hard drive grid. 

In [65]:
input = 'hfdlxzhv'
grid = {}

#make sure hash is still working
assert dense_hash(sparse_hash(256,"AoC 2017"))  == "33efeb34ea91902bb2f59c9920caa6cd"

#generate hashes
for i in range(0,128):
    row = ""
    hash = dense_hash(sparse_hash(256,input + "-" +str(i)))
    
    for hex in hash:
        row = row + (bin(int(hex, 16))[2:].zfill(4))
    grid[i] = [int(x) for x in row]
    
    
#count number of positions used
squares_used = 0

for row in grid.keys():
    squares_used += sum(grid[row])
    
print(squares_used)

8230


###Part 2:
We scan through the grid until we encounter an occupied square.  We than empty that square out and all connected squares to get the grouping.  We continue this until we have scanned through all squares in the grid.

In [66]:
regions = 0

#grid = {}
#grid[0] = [0,0,1,1]
#grid[1] = [1,0,0,1]
#grid[2] = [0,1,0,1]
#grid[3] = [1,0,0,1]


rows = 128
cols = 128
for row in range(0,rows):
    for col in range(0,cols):
        stack = []
    
        #if we encounter an occupied square
        if grid[row][col] == 1:
            #increment regions
            regions += 1
            grid[row][col] = 0
            stack.append([row,col])
        
        #clear out all adjacent squares using a stack
        while stack:
            c_row,c_col = stack.pop()
            #check above
            if c_row - 1 >= 0:
                if grid[c_row-1][c_col] == 1:
                    grid[c_row-1][c_col] = 0
                    stack.append([c_row-1,c_col])
                    
            #check below
            if c_row + 1 <= rows - 1:
                if grid[c_row+1][c_col] == 1:
                    grid[c_row+1][c_col] = 0
                    stack.append([c_row+1,c_col])
    
            #check right
            if c_col + 1 <= cols - 1:
                if grid[c_row][c_col+1] == 1:
                    grid[c_row][c_col+1] = 0
                    stack.append([c_row,c_col+1])
                    
            
            #check left
            if c_col - 1 >= 0:
                if grid[c_row][c_col-1] == 1:
                    grid[c_row][c_col-1] = 0
                    stack.append([c_row,c_col-1])
                
print(regions)

1103


##Day 15: Dueling Generators

###Part 1:
Simple brute force for loop generating numbers and counting pairs.  The bitwise AND operation with 0xFFFF sped up the process alot vs doing string manipulation!

In [69]:
#Constants for problem
gen_a = 703
gen_b = 516

gen_a_factor = 16807
gen_b_factor = 48271

divisor  = 2147483647

matches = 0
a_queue = []
b_queue = []
#loop
for i in range(0,40000000):
    gen_a*= gen_a_factor
    gen_a = gen_a % divisor
    
    gen_b *= gen_b_factor
    gen_b = gen_b % divisor
    
    #Get the int equivalent to the last 16 bits of the numbers
    gen_a_bin = gen_a & 0xFFFF
    gen_b_bin = gen_b & 0xFFFF
    
    if gen_a_bin == gen_b_bin:
        matches += 1
        

print(matches)

594


###Part 2:

Try implementing with generators to compare the pairs.  There is a seperate generator for each.  We stop after 5,000,000 comparisons.

In [70]:
def a_gen():
    a = 703
    while True:
        a  *=16807
        a  %= 2147483647
    
        if a % 4 == 0:
            yield a
        
def b_gen():
    b = 516
    while True:
        b  *=48271
        b  %= 2147483647
    
        if b % 8 == 0:
            yield b
        
# perform 5 million comparisons
match = 0
a = a_gen()
b = b_gen()

for _ in range(0,5000000):
    cur_a = next(a) & 0xFFFF
    cur_b = next(b) & 0xFFFF
    if cur_a == cur_b:
        match +=1
        
print(match)

328


##Day 16: Permutation Promenade

###Part 1:
Create an array of the programs.  Then perform "dance" by performing each command in the list.

In [70]:
#generate array
def rotate_prog(prog,n):
    rotate = -(n%len(prog))
    return prog[rotate:] + prog[:rotate]

def swap_pos(prog,A,B):
    temp=prog[A]
    prog[A] = prog[B]
    prog[B] = temp
    return prog

def swap_prog(prog,A,B):
    index_a = prog.index(A)
    index_b = prog.index(B)
    temp=prog[index_a]
    prog[index_a] = prog[index_b]
    prog[index_b] = temp
    return prog
    
prog = list("abcdefghijklmnop")

#load data
commands = load_data('day16').read()
commands = commands.rstrip().split(",")


 
for command in commands:
    if command[0] == 's':
        prog = rotate_prog(prog,int(command[1:]))
    elif command[0] == 'p':
        prog = swap_prog(prog,command[1],command[3])
    elif command[0] == 'x':
        pos = command[1:].split("/")
        prog = swap_pos(prog,int(pos[0]),int(pos[1]))
        
        
print("".join(prog))

olgejankfhbmpidc


###Part 2:

The output list of programs will start to cycle after X number cycles.  We just need to find each value of the program list at each point in the cycle.  1 billion % the cycle length will give us which location in the cycle the "dance" will end upon.  By doing this we avoid having to run all 1 billion iterations!

In [71]:
prog = list("abcdefghijklmnop")

#load data
commands = load_data('day16').read()
commands = commands.rstrip().split(",")
memo = {}

#Find number of repeat cycles
encountered_set = set()
cycle = []


while ''.join(prog) not in encountered_set:
    encountered_set.add(''.join(prog))
    cycle.append(''.join(prog))
      
    for command in commands:
        if command[0] == 's':
            prog = rotate_prog(prog,int(command[1:]))
        elif command[0] == 'p':
            prog = swap_prog(prog,command[1],command[3])
        elif command[0] == 'x':
            pos = command[1:].split("/")
            prog = swap_pos(prog,int(pos[0]),int(pos[1]))
            
        
print(int(1e9%len(cycle)))
print(cycle[int(1e9%len(cycle))])

40
gfabehpdojkcimnl


##Day 17: Spinlock

###Part 1:
The spinlock function performs the spinlock steps as specified in the problem.

In [72]:
def spinlock(step_size,steps):
    arr = [0]
    ptr = 0
    for step in range(1,(steps+1)):
        ptr = ((ptr)+step_size) % (len(arr))
        arr = arr[:(ptr+1)] + [step] + arr[(ptr+1):]
        ptr+=1
    return(arr[ptr+1])
    
assert spinlock(3,2017) == 638
spinlock(316,50)

45

###Part 2:
The first position in the array will always be zero. So the value to the right of zero will be the value in position 1 in the array after 50,000,000 cycles.  So we don't need to maintain and update the full array the entire time, but just track when the ptr is at position 0.

In [73]:
val = 0
ptr=0
step_size = 316
for step in range(1,(50000000+1)):
    ptr = ((ptr)+step_size) % (step)
    if ptr == 0:
        val = step
    ptr +=1
        
print(val)

13326437


##Day 18: Duet

###Part 1:

First load data than cycle through performing instructions until we hit 'rcv'.  At which point break and return the last sound played. 

In [74]:
#load data
input = load_data('day18')
cmd= []
reg = {}

for line in input:
    nc = line.rstrip().split(" ")
    #add in place holder for lines without a value
    if len(nc) == 2:
        nc.append('0')
    cmd.append(nc)
    reg[nc[1]] = 0

#process
cmd_ptr = 0
sound = 0

while True:
    instruct, register, value = cmd[cmd_ptr]

    #check if value is another register
    if ord(value[0]) >= 97 or ord(value[0]) >= 122:
        value=reg[value]
        
    #perform commands
    if instruct == 'set':
        reg[register] = int(value)
        
    elif instruct =='mul':
        reg[register] *= int(value)
        
    elif instruct == 'add':
        reg[register] += int(value)
        
    elif instruct == 'mod':
        reg[register] = reg[register] % int(value)
        
    elif instruct == 'jgz' and reg[register] > 0:
        cmd_ptr += int(value)
        continue
        
    elif instruct =='snd':
        sound = reg[register]
        
    elif instruct =='rcv' and reg[register] != 0:
        print(sound)
        break
        
    cmd_ptr +=1

3423


###Part 2:

This was a little bit tricker.  We run the two programs independantly.  Run the 1st until it is deadlocked and needs to receive from the second.  While the 1st is running we place it's sends into a queue.  We than repeat the process for program 2.  We iterate back and forth between the two programs until they are both deadlocked and cannot proceed. 

In [75]:
reg_0 = {}
reg_1 = {}
cmd = []

#load data
input = load_data('day18')
for line in input:
    nc = line.rstrip().split(" ")
    #add in place holder for lines without a value
    if len(nc) == 2:
        nc.append('0')
    cmd.append(nc)
    
    if ord(nc[1]) >= 97 or ord(nc[1]) >= 122:
        reg_0[nc[1]] = 0
        reg_1[nc[1]] = 0
    
#initialize everything
reg_0['p'] = 0
reg_1['p'] = 1
ptr_0 = 0
ptr_1 = 0
queue_0 = []
queue_1 = []

#cycle through and run each program until you can't anymore
def cycle(reg, ptr, queue_in, queue_out,cmd):
    snd_count = 0
    count = 0
    while True:
        instruct, register, value = cmd[ptr]
        #print(cmd[ptr])
        #check if value is another register
        if ord(value[0]) >= 97 or ord(value[0]) >= 122:
            value=reg[value]
            
        #perform commands
        if instruct == 'set':
            reg[register] = int(value)
            
        elif instruct =='mul':
            reg[register] *= int(value)
            
        elif instruct == 'add':
            reg[register] += int(value)
            
        elif instruct == 'mod':
            reg[register] = reg[register] % int(value)
            
        elif instruct == 'jgz':
            
            #can be a register or int
            if ord(register[0]) >= 97 or ord(register[0]) >= 122:
                register=reg[register]
                
            if int(register) > 0:    
                ptr += int(value)
                continue
            
        elif instruct =='snd':
            queue_out.append(reg[register])
            snd_count +=1
            
        elif instruct =='rcv':
            if queue_in:
                reg[register] = queue_in.pop(0)
            else:
                break
            
        ptr +=1

    return reg, ptr, queue_in, queue_out, snd_count

#keep cycling both programs until dead locked
sends = 0
sends_tot = 0


while True:
    reg_0, ptr_0, queue_0, queue_1, sends = cycle(reg_0, ptr_0, queue_0, queue_1,cmd)
    reg_1, ptr_1, queue_1, queue_0, sends = cycle(reg_1, ptr_1, queue_1, queue_0,cmd)
    sends_tot += sends
    if len(queue_0) == 0 and len(queue_1) == 0:
        break
        
#return count
print(sends_tot)


7493


##Day 19: A Series of Tubes

###Part 1/Part 2:

Can solve part 1 and two together.  Basically the packed always continues in the same direction until a '+' is encountered.  At which point we must scan around the '+' to find a '-' or '|' to determine which direction to change.  After a packet travel through a position the path is removed.  Whenever we encounter a letter we record it for the solution.

In [77]:
#load data
input = load_data('day19')
tubes = []
x = 0
y = 0
#term to keep track of our momentum up,down,left,right
momentum = [1,0] 
#term to keep track of previous char
prev_char = "|"

#order of letters
letters = []

for line in input:
    tubes.append(list(line.rstrip('\n')))

tubes.append([' ' for x in range(0,len(tubes[-1]))])

#find out start coordinate
for point in range(0,len(tubes[0])):
    if tubes[x][point] != " ":
        y = point



count = 0
while len(letters) < 10:
    current_char = tubes[x][y]
    tubes[x][y] = " "
    
    #if letter    
    if current_char in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
        letters.append(current_char)
        
    #if '+' we need to update momentum
    elif current_char == '+':
        if tubes[x+1][y] != " ":
            momentum = [1,0]
            
        elif tubes[x-1][y] != " ":
            momentum = [-1,0]
            
        elif tubes[x][y-1] != " ":
            momentum = [0,-1]
            
        elif tubes[x][y+1] != " ":
            momentum = [0,1]
        
    x_diff, y_diff = momentum
    x += x_diff
    y += y_diff
    count +=1
    
print(''.join(letters))
print("Steps taken: " + str(count))


EPYDUXANIT
Steps taken: 17544


##Day 20: Particle Swarm

###Part 1:
Create three seperate dicts for acceleration, velocity and particle position.  For each tick we add acceleration to velocity and velocity to position.   I just ran the ticks for an arbitrary large number and than found the nearest particle to the origin.

In [78]:
#load data
input = load_data('day20')
particles = []
velocity = []
acel = []
p_count = 0

#process lists for processing
for line in input:
    p,v,a = line.rstrip().split(", ")
    particles.append(list(map(int,p[3:-1].rstrip().split(","))))
    velocity.append(list(map(int,v[3:-1].rstrip().split(","))))
    acel.append(list(map(int,a[3:-1].rstrip().split(","))))

#lets assume long term steady state at 1000 'ticks' 
for i in range(0,500):
    collisions = {}
    for particle in range(0,len(particles)):
        velocity[particle] = list(map(operator.add,velocity[particle], acel[particle]))
        particles[particle] = list(map(operator.add,particles[particle], velocity[particle]))


#find nearest particle
min_particle_index = 0
min_particle_distance = 9999999999

for i in range(0,len(particles)):
    dist = abs(particles[i][0]) + abs((particles[i][1])) + abs((particles[i][2])) 
    if dist < min_particle_distance:
        min_particle_distance = dist
        min_particle_index = i
        
print(min_particle_index)

144


###Part 2:
Used modified code from part 1.  At each tick check for a collision between particles.  Any collised particles are labelled with 'c'.

In [79]:
#load data
input = load_data('day20')
particles = []
velocity = []
acel = []
p_count = 0

#process lists for processing
for line in input:
    p,v,a = line.rstrip().split(", ")
    particles.append(tuple(map(int,p[3:-1].rstrip().split(","))))
    velocity.append(tuple(map(int,v[3:-1].rstrip().split(","))))
    acel.append(tuple(map(int,a[3:-1].rstrip().split(","))))

#lets assume long term steady state at 1000 'ticks' 
for i in range(0,1000):
    for particle in range(0,len(particles)):
        if particles[particle] !='c':
            velocity[particle] = tuple(map(operator.add,velocity[particle], acel[particle]))
            particles[particle] = tuple(map(operator.add,particles[particle], velocity[particle]))
    
    #check for collisions
    
    #get list of collisions
    collisions = set()
    locations = Counter(particles)
    for location in locations:
        if locations[location] > 1:
            collisions.add(location)       
        
    #update collided values
    for particle in range(0,len(particles)):
        if particles[particle] in collisions:
            particles[particle] = 'c'
            velocity[particle] = 'c'
            acel[particle] = 'c'

#count particles left
remaining = 0
for particle in particles:
    if particle != 'c':
        remaining+=1
        
remaining

477

##Day 21: Fractal Art

###Part 1/Part 2:

This one was pretty long.  I created a dict of the conversions for the enhancements to be able to quickly look them up.  For each enchancment input I pre-generated all the transforms(flip/rotate) and put them into this dict for lookup.  We then look at the grid and apply the enhancements to build up a new grid.

In [80]:
#load enhancement book and put into dict
input = load_data('day21')

#dict for enhancements
eb = {}

#generates all rotations and flips of an array
def transforms(pattern):
    res = []
    for i in range(0,4):
        tp = np.rot90(pattern, k = i)
        res.append(tp)
        res.append((np.fliplr(tp)))
        res.append((np.flipud(tp)))
        
    return res
        
#generate the full enhancement book
for line in input:
    start,out = line.rstrip().split(" => ")
    
    start = start.replace("/", "")
    out = out.replace("/", "")

    if len(start) == 4:
        size =2
    else:
        size = 3
    
    start = np.array([1 if x == "#" else 0 for x in start]).reshape((size,size))
    out = np.array([1 if x == "#" else 0 for x in out]).reshape((size+1,size+1))

    for transform in transforms(start):
        eb[transform.tostring()] = out

#make sure book is working       
assert (eb[np.array([0,1,0,0,0,1,1,1,1]).reshape((3,3)).tobytes()]==np.array([1,0,1,1,1,0,1,0,0,0,1,0,0,0,1,1]).reshape(4,4)).all()

#time to run!
grid= np.array(np.array([0,1,0,0,0,1,1,1,1]).reshape(3,3))
size = 3

for k in range(0,18):
    #if current grid divisible by two
    if size % 2 ==0:
        newsize = size // 2 * 3
        new_grid = np.empty((newsize,newsize),dtype=int)
        #row count
        for i in range(0,size,2):
            #col count
            for j in range(0,size,2):
                new_grid[i//2*3:i//2*3+3,j//2*3:j//2*3+3] = eb[grid[i//2*2:i//2*2+2,j//2*2:j//2*2+2].tobytes()]
    #if current grid divisible by three        
    else:
        newsize = size // 3 * 4
        new_grid = np.empty((newsize,newsize),dtype=int)
        #row count
        for i in range(0,size,3):
            #col count
            for j in range(0,size,2):
                new_grid[i//3*4:i//3*4+4,j//3*4:j//3*4+4] = eb[grid[i//3*3:i//3*3+3,j//3*3:j//3*3+3].tobytes()]
                
    grid = new_grid
    size = newsize
    
np.sum(grid)

1879071

##Day 22: Sporifica Virus

###Part 1:

To track the direction the various is tracking I used a numpy array: [[1,0],[0,-1]].  By rotating this array it will tell us which way the virus will currently need to move.  The array can be easily rotated is np.rot90.  We also need to resize the grid whenever we get to its edge.

In [81]:
#load grid and put into numpy grid
grid = []
input = load_data('day22')

for row in input:
    grid.append([1 if x =="#" else 0 for x in list(row.rstrip())])

#convert to numpy    
grid = np.array(grid)

#initalize virus locations
v_x = grid.shape[0]//2
v_y = grid.shape[1]//2

#direction matrix for virus 
v_directions = np.array([[0,-1],[1,0]])

#start facing up
v_cur = 2
infect_count = 0
for iteration in range(0,10000):
    #if current grid infected
    if grid[v_x,v_y] == 1:
        #turn right
        v_directions = np.rot90(v_directions, k = -1)
        #clean
        grid[v_x,v_y] = 0
        
    #if uninfected:
    else:
        #turn left
        v_directions = np.rot90(v_directions, k = 1)
        #infect
        grid[v_x,v_y] = 1
        infect_count+=1
        
    v_x += v_directions[0,1]
    v_y += v_directions[0,0]
    
    #make grid larger if necessary
    if (v_x < 0 or v_x >= grid.shape[0]) or (v_y < 0 or v_y >= grid.shape[1]):
        #pad grid
        grid = np.vstack((np.zeros((1,grid.shape[1]),dtype=int),grid,np.zeros((1,grid.shape[1]),dtype=int)))
        grid = np.hstack((np.zeros((grid.shape[0],1),dtype=int),grid,np.zeros((grid.shape[0],1),dtype=int)))
        v_x+=1
        v_y+=1
        
infect_count

5462

###Part 2:

Modified code from part 1 to include the new weakened and flagged states.

In [82]:
#load grid and put into numpy grid
#0 = clean
#1 = infected
#2 = weak
#3=flagged
grid = []
input = load_data('day22')

for row in input:
    grid.append([1 if x =="#" else 0 for x in list(row.rstrip())])

#convert to numpy    
grid = np.array(grid)

#initalize virus locations
v_x = grid.shape[0]//2
v_y = grid.shape[1]//2

#direction matrix for virus 
v_directions = np.array([[0,-1],[1,0]])

#start facing up
v_cur = 2
infect_count = 0
for iteration in range(0,10000000):
    #if current grid infected
    if grid[v_x,v_y] == 1:
        #turn right
        v_directions = np.rot90(v_directions, k = -1)
        #clean
        grid[v_x,v_y] = 3
        
    #if uninfected:
    elif grid[v_x,v_y] == 0:
        #turn left
        v_directions = np.rot90(v_directions, k = 1)
        #weaken
        grid[v_x,v_y] = 2
        
        #if weak:
    elif grid[v_x,v_y] == 2:
        #no turn
        #infect
        grid[v_x,v_y] = 1
        infect_count +=1
        
    else:
        #rotate 180
        v_directions = np.rot90(v_directions, k = 2)
        #clean
        grid[v_x,v_y] = 0

        
    v_x += v_directions[0,1]
    v_y += v_directions[0,0]
    
    #make grid larger if necessary
    if (v_x < 0 or v_x >= grid.shape[0]) or (v_y < 0 or v_y >= grid.shape[1]):
        #pad grid
        grid = np.vstack((np.zeros((1,grid.shape[1]),dtype=int),grid,np.zeros((1,grid.shape[1]),dtype=int)))
        grid = np.hstack((np.zeros((grid.shape[0],1),dtype=int),grid,np.zeros((grid.shape[0],1),dtype=int)))
        v_x+=1
        v_y+=1
        
infect_count

2512135

##Day 23: Coprocessor Conflagration

###Part 1:

I reused/updated some of the code from day 18.  It's similar in following commands to update register values.

In [83]:
#load data
input = load_data('day23')
cmd= []
reg = {'a':0,
       'b':0,
       'c':0,
       'd':0,
       'e':0,
       'f':0,
       'g':0,
       'h':0}

for line in input:
    nc = line.rstrip().split(" ")
    cmd.append(nc)

#process
cmd_ptr = 0
mul_count = 0

while True:
    if cmd_ptr >= len(cmd):
        break
        
    instruct, register, value = cmd[cmd_ptr]
    #check if value is another register
    if ord(value[0]) >= 97 and ord(value[0]) <= 122:
        value=reg[value]
    
    #check if reg value is an int
    if ord(register[0]) >= 97 and ord(register[0]) <= 122:
        reg_value=reg[register]
    else:
        reg_value = register
        
    #perform commands
    if instruct == 'set':
        reg[register] = int(value)
        
    elif instruct =='mul':
        reg[register] *= int(value)
        mul_count+=1
        
    elif instruct == 'sub':
        reg[register] -= int(value)

    elif instruct == 'jnz' and int(reg_value) != 0:
        cmd_ptr += int(value)
        continue
        
        
    cmd_ptr +=1
    
print(mul_count)

6241


###Part 2:
This one was tricky!  This video was very [helpful](https://www.youtube.com/watch?v=AqXTZo6o34s).

Essentially this is a giant loop that is incrementing between 108,100 and 125,100 by 17 and counting the number of non-prime numbers seen.

In [84]:
h = 0
for i in range(108100,125101,17):
    for x in range(2,i):
        if i % x == 0:
            h+=1
            break
            
print(h)

909


##Day 24: Electromagnetic Moat

###Part 1:
You can treat the bridge components are edges in a undirected graph.  We can than use DFS to find all the potential paths in the graph and determine the one with the greatest sum.  I use dfs_path with a stack to perform dfs.

In [85]:
#load data
input = load_data('day24')
comps = {}

for line in input:
    l,r = line.rstrip() .split("/")
    comps.setdefault(int(l),set()).add(int(r))
    comps.setdefault(int(r),set()).add(int(l))


#remove self referencing edges
#for num in comps.keys():
#    if num in comps[num]:
#        comps[num].remove(num)
    
def dfs_path(graph,start):
    paths = []
    stack = []
    stack.append([start,[start]])
    while stack:
        node, visited = stack.pop()
        for edge_end in graph[node[1]]:
            edge = [node[1],edge_end]
            if edge not in visited and edge[::-1] not in visited:
                paths.append(visited + [edge])
                stack.append([edge,visited + [edge]])
    return paths

max_path =0
#paths = dfs_path(comps,[0,2]) + dfs_path(comps,[0,1])
paths = dfs_path(comps,[0,9]) + dfs_path(comps,[0,7])
for path in paths: 
    if sum(list(map(sum,path))) > max_path:
        max_path =sum(list(map(sum,path)))
        
max_path
        

1868

###Part 2:
Take the results from part 1 with gives us all the potential paths, to find the longest path.


In [86]:
max_length = 0
max_path = 0
for path in paths:
        if len(path) >= max_length:
            max_path = sum(list(map(sum,path)))
            max_length = len(path)
            
print(max_path)
print(max_length)

1841
40


##Day 25: Turing Machine

###Part 1:
We are using commands in English to modify values in a list.  This is mainly about processing text to determine how to perform the commands.  I just hardcoded the location of important numbers/letters in text for the commands.

In [87]:
#load data
input = load_data('day25')
turing = []
tape = [0] * 1000000
ptr = len(tape)//2

for line in input:
    turing.append(line.rstrip('\n').lower())

#get start state and number of steps    
current_state = turing[0][-2]
total_steps = int(turing[1].split(" ")[5])
turing = turing[2:]

states ={}

def get_dir(input):
    if input.split(" ")[-1] == 'right.':
        return 1
    else:    
        return -1
    
    
for i in range(0,len(turing),10):
    state = turing[i+1][-2]
    #format: what to write, direction to move,next state.  For 0 (first array) and for 1 (second array)
    states[state] = [[int(turing[i+3][-2]), get_dir(turing[i+4]),turing[i+5][-2]],
                     [int(turing[i+7][-2]), get_dir(turing[i+8]),turing[i+9][-2]]]
    
for step in range(0,total_steps):
    cur_value = tape[ptr]
    tape[ptr] = states[current_state][cur_value][0]
    ptr += states[current_state][cur_value][1]
    current_state = states[current_state][cur_value][2]
    
print(sum(tape))

4387
